In [23]:
import torch
import torch.nn

def corr2d(X, k):
    n_w, n_h = X.shape
    k_w, k_h = k.shape
    ret_mat = torch.zeros((n_w - k_w + 1, n_h - k_h + 1))
    for i in range(n_w - k_w + 1):
        for j in range(n_h - k_h + 1):
            ret_mat[i, j] = (X[i:i+k_w, j:j+k_h] * k).sum()
    return ret_mat

# multiple input channel, 1 output channel
# input: (ci, a0, a1)
# kernel: (ci, k0, k1)
# output: (a0-k0+1, a1-k0+1)
def corr2d_multi_in(X, K):
    return sum([corr2d(x, k) for x,k in zip(X, K)])
    
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]], 
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], 
                  [[1.0, 2.0], [3.0, 4.0]]])

print(X.shape, K.shape)
print(corr2d_multi_in(X, K))

torch.Size([2, 3, 3]) torch.Size([2, 2, 2])
tensor([[ 56.,  72.],
        [104., 120.]])


In [27]:
# multiple input channel(ci), multiple output channel(co)
# input: (ci, a0, a1)
# kernel: (co, ci, k0, k1)
# output: (co, a0-k0+1, a1-k0+1)
def corr2d_multi_in_multi_out(X, K):
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

K_m = torch.stack((K, K+1, K+2), 0)
print(X.shape, K_m.shape)
print(corr2d_multi_in_multi_out(X, K_m))

torch.Size([2, 3, 3]) torch.Size([3, 2, 2, 2])
tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])


In [34]:
# special case of corr2d_multi_in_multi_out: when kernel_size = (1), equals fully connect layers
# input: (ci, a0, a1)
# kernel: (co, ci, k0, k1) = (co, ci, 1, 1)
# output: (co, a0-k0+1, a1-k1+1) = (co, a0-1+1, a1-1+1) = (co, a0, a1)
def corr2d_multi_in_multi_out_1x1(X, K):
    ci, a0, a1 = X.shape
    co = K.shape[0]
    X = X.reshape(( ci, -1 ))
    K = K.reshape(( co, -1 ))
    return torch.matmul(K, X).reshape((co, a0, a1))

X = torch.arange(27, dtype=torch.float32).reshape(3,3,3)
Y = torch.ones(6, dtype=torch.float32).reshape(2,3,1,1)

print(X.shape, Y.shape)
print(corr2d_multi_in_multi_out_1x1(X, Y))
print(corr2d_multi_in_multi_out(X, Y))

torch.Size([3, 3, 3]) torch.Size([2, 3, 1, 1])
tensor([[[27., 30., 33.],
         [36., 39., 42.],
         [45., 48., 51.]],

        [[27., 30., 33.],
         [36., 39., 42.],
         [45., 48., 51.]]])
tensor([[[27., 30., 33.],
         [36., 39., 42.],
         [45., 48., 51.]],

        [[27., 30., 33.],
         [36., 39., 42.],
         [45., 48., 51.]]])
